# 03. Position-wise Feed-Forward Network 구현

## 학습 목표
- FFN의 구조와 역할 이해
- "Position-wise"의 의미 이해
- ReLU vs GELU 차이

## 관련 면접 질문
- Transformer에서 FFN의 역할은?
- Position-wise가 무슨 뜻인가요?
- d_ff는 보통 d_model의 몇 배인가요?

## 핵심 수식
$$\text{FFN}(x) = \text{Activation}(xW_1 + b_1)W_2 + b_2$$

---
## Pre-Quiz

### Q1. FFN에서 d_ff가 d_model보다 큰 이유는?
```
당신의 답: 
```

### Q2. "Position-wise"가 의미하는 것은?
```
당신의 답: 
```

### Q3. BERT/GPT에서 ReLU 대신 GELU를 쓰는 이유는?
```
당신의 답: 
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False
torch.manual_seed(42)

---
## 1. FFN 구현

### 구조
```
Input: (batch, seq, d_model)
    ↓ Linear(d_model → d_ff)
(batch, seq, d_ff)
    ↓ Activation (GELU/ReLU)
    ↓ Dropout
    ↓ Linear(d_ff → d_model)
    ↓ Dropout
Output: (batch, seq, d_model)
```

In [ ]:
# =================================================
# TODO: Position-wise FFN 구현
# =================================================
#
# Args:
#   d_model: 입력/출력 차원
#   d_ff: hidden 차원 (default: d_model * 4)
#   dropout: dropout 확률
#   activation: 'relu' 또는 'gelu'
#
# 구조:
#   fc1: Linear(d_model, d_ff)
#   fc2: Linear(d_ff, d_model)
#   activation: GELU 또는 ReLU
#
# =================================================

class PositionWiseFeedForward(nn.Module):
    def __init__(
        self,
        d_model: int,
        d_ff: int = None,
        dropout: float = 0.1,
        activation: str = 'gelu'
    ):
        super().__init__()
        if d_ff is None:
            d_ff = d_model * 4
        
        # ============================================
        # 여기에 구현하세요
        # fc1, fc2, dropout, activation 함수
        # ============================================
        
        pass  # 이 줄을 지우고 구현
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: (batch, seq, d_model)
        Returns:
            output: (batch, seq, d_model)
        """
        # ============================================
        # 여기에 구현하세요
        # x → fc1 → activation → dropout → fc2 → dropout
        # ============================================
        
        pass  # 이 줄을 지우고 구현

In [ ]:
# ============ 테스트 ============
def test_ffn():
    batch, seq, d_model = 2, 10, 64
    
    ffn = PositionWiseFeedForward(d_model, d_ff=256, activation='gelu')
    x = torch.randn(batch, seq, d_model)
    
    output = ffn(x)
    
    assert output.shape == x.shape, f"Shape 오류: {output.shape}"
    print("✅ FFN 테스트 통과!")
    print(f"   Input: {x.shape}")
    print(f"   Output: {output.shape}")
    
    # 파라미터 수 확인
    total_params = sum(p.numel() for p in ffn.parameters())
    print(f"   Total parameters: {total_params:,}")

test_ffn()

---
## 2. ReLU vs GELU 비교

In [ ]:
x = torch.linspace(-3, 3, 100)

relu = F.relu(x)
gelu = F.gelu(x)

plt.figure(figsize=(10, 5))
plt.plot(x.numpy(), relu.numpy(), label='ReLU', linewidth=2)
plt.plot(x.numpy(), gelu.numpy(), label='GELU', linewidth=2)
plt.axhline(y=0, color='black', linewidth=0.5)
plt.axvline(x=0, color='black', linewidth=0.5)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('ReLU vs GELU\nGELU는 음수 영역에서도 작은 값 통과 (smoother)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---
## Post-Quiz

### Q1. d_model=512, d_ff=2048일 때 FFN의 파라미터 수는?
```
당신의 답: (계산식 포함)
```

### Q2. FFN이 없으면 Transformer가 어떻게 되나요?
```
당신의 답: 
```

---
## 정답

<details>
<summary>👉 클릭하여 정답 보기</summary>

### Pre-Quiz

**Q1. d_ff가 큰 이유**
> - 확장 후 축소로 **비선형 변환 능력** 증가
> - 더 복잡한 패턴 학습 가능
> - "bottleneck" 아키텍처의 역방향

**Q2. Position-wise 의미**
> - 각 position에 **독립적으로** 적용
> - 같은 weight를 공유하지만 위치별로 따로 계산
> - CNN의 1x1 convolution과 유사

**Q3. GELU 사용 이유**
> - ReLU보다 smooth → gradient 흐름 좋음
> - 음수에서도 작은 값 통과 (Dying ReLU 방지)
> - 실험적으로 NLP에서 더 나은 성능

---

### Post-Quiz

**Q1. 파라미터 수**
> - fc1: 512 × 2048 + 2048 = 1,050,624
> - fc2: 2048 × 512 + 512 = 1,049,088
> - **Total: 약 210만 개**

**Q2. FFN 없으면?**
> - Attention만으로는 선형 변환에 가까움
> - **비선형성 부족** → 표현력 감소
> - FFN이 각 토큰에 비선형 변환 적용

</details>

---
## 구현 정답

<details>
<summary>👉 클릭하여 정답 코드 보기</summary>

```python
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff=None, dropout=0.1, activation='gelu'):
        super().__init__()
        if d_ff is None:
            d_ff = d_model * 4
        
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        
        if activation == 'gelu':
            self.activation = F.gelu
        else:
            self.activation = F.relu
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x
```

</details>